# IoTアプリケーションの開発効率の向上を目的とした開発者によるコードの変更を部分適用する方式の提案と実装

このノートブックは、研究報告「IoTアプリケーションの開発効率の向上を目的とした開発者によるコードの変更を部分適用する方式の提案と実装」における方式の検証を目的とするものである。

Nervesを用いて実装したIoTアプリケーションについて、開発者によるコードの変更をIoTデバイスへ適用する方式を比較検討する。

計測方法について、北九州市立大学の山崎進氏による「[mix_tasks_upload_hotswap の Hexライブラリ版を試す](https://qiita.com/zacky1972/items/f0b47eded7c902008871)」がおおいに参考になりました。感謝いたします。

## 対象とするIoTアプリケーション

各方式の比較検討に用いるアプリケーションは、以下の通りである。`TargetApp.hello/0`メソッドが返す文字列を変更する。

```
defmodule TargetApp do
  def hello do
    :world
  end
end
```

## 比較検討する方式

1. ファームウェアイメージ全体の適用
2. ファームウェアイメージのパッチによる部分適用
3. アプリケーションコードの部分適用

それぞれの詳細については、研究報告論文を参照のこと。

## 比較検討の方法

方式の比較に用いる指標は、開発者によるコードの変更をIoTデバイスに適用し、アプリケーションが動作するまでの時間を用いる。

計測には、1.および2.についてはスクリプト[./measure_firmware_update.sh](./measure_firmware_update.sh)を用い、3.については`mix upload.hotswap`コマンドの実行を`time(1)`によって計測する。

```
#!/bin/sh

BUILD_METHOD=$1

if [ -z "$BUILD_METHOD" ]
then
  echo "usage: $0 [firmware | firmware.patch]"
  exit 1
fi

set -u

IP_ADDRESS=$(ping -c 1 nerves.local | grep 'PING nerves.local' | perl -nle 's/^.+\((\d+\.\d+\.\d+\.\d+)\).+$/$1/; print')
echo "nerves.local is at $IP_ADDRESS"
echo ""

echo "Time to build firmware (mix $BUILD_METHOD)"
echo "======================================"

time mix "$BUILD_METHOD" > /dev/null
echo ""

echo "Time to upload firmware"
echo "======================================"

if [ "$BUILD_METHOD" = "firmware" ]
then
  time mix upload > /dev/null
else
  time mix upload --firmware _build/rpi3_dev/nerves/images/patch.fw > /dev/null
fi
echo ""

echo "Time to reboot"
echo "======================================"
echo ""

START_TIME=$(gdate +"%s.%3N")

# 再起動に入るまでしばらく時間がかかる。
# その間はpingは通るので、通らなくなるまでpingし続ける。
ping -c 1 -t 1 "$IP_ADDRESS" > /dev/null

# pingが通らなくなったら終了ステータスが2になる
while [ $? -ne 2 ] 
do
  ping -c 1 -t 1 -i 0.1 "$IP_ADDRESS" > /dev/null
done

# pingが通らなくなったら再起動中ということなので、
# あらためてタイムアウトを長く設定してpingする
ping -c 1 -t 100 "$IP_ADDRESS" > /dev/null

END_TIME=$(gdate +"%s.%3N")
DURATION=$(echo "scale=1; $END_TIME - $START_TIME" | bc)
echo "time: $DURATION sec."
echo ""
```

### 1. ファームウェアイメージ全体の適用

1. `mix firmware`
2. `mix upload`
3. Nervesが再起動して`ping(1)`が通るようになるまでの時間

を計測する。

In [4]:
!cd target_app && ../measure_firmware_update.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m21.927s
user	0m6.584s
sys	0m2.121s

Time to upload firmware

real	0m14.572s
user	0m5.182s
sys	0m1.261s

Time to reboot

time: 22.692 sec.



### 2. ファームウェアイメージのパッチによる部分適用

1. `mix firmware.patch`
2. `mix upload --firmware _build/rpi3_dev/nerves/images/patch.fw`
3. Nervesが再起動して`ping(1)`が通るようになるまでの時間

を計測する。

In [8]:
!cd target_app && ../measure_firmware_update.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m27.127s
user	0m6.697s
sys	0m2.227s

Time to upload firmware

real	0m17.873s
user	0m3.897s
sys	0m0.952s

Time to reboot

time: 22.534 sec.



### 3. アプリケーションコードの部分適用

1. `mix upload.hotswap`

を計測する。この方式ではNervesの再起動は必要ないため、適用の手順は以上のみである。

In [9]:
!cd target_app && time mix upload.hotswap > /dev/null

** (Mix) The task "upload.hotswap" could not be found. Did you mean "upload"?

real	0m2.957s
user	0m4.714s
sys	0m1.225s
